* **FA17-BL-INFO-I590-14120**
* **Data Visualization**
* **Gerald Manipon, MS Data Science student - Indiana University**
* **gmanipon@iu.edu**
* **this jupyter notebook can be found here: https://github.com/pymonger/tropicalstorm-data-visualization**

# Requirements

1. Install anaconda3 (https://www.continuum.io/downloads)
1. Install requisite packages:
   ```
   conda install netCDF4
   conda install pandoc
   conda install -c conda-forge ipyleaflet
   conda install -c conda-forge gdal
   conda install -c conda-forge astropy
   ```
1. Add the following line to your .bash_profile and source it:
   ```
   export GDAL_DATA=$(gdal-config --datadir)
   ```
1. Download source dataset to directory containing this notebook file:
   ```
   wget ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r10/wmo/netcdf/Allstorms.ibtracs_wmo.v03r10.nc
   ```
1. Run jupyter:
   ```
   jupyter notebook
   ```

<font color='red'>**WARNING: If the above packages are not installed or the source dataset not downloaded, you will not be able to parse the NetCDF4 dataset nor will you be able to render leaflet visualizations used in this notebook.**</font>

# Introduction

As a native of the Hawaiian Islands, I’ve lived through two hurricanes, Hurricane Iwa in 1982 and Hurricane Iniki in 1992, with a 10-year span between their occurrences. Since 1992, there have been a number of hurricanes that have come close to the islands but none that have made an impact like that of Iwa with a damage cost of \$773 million dollars or Iniki with a damage cost of \$3.2 billion dollars (both adjusted to 2006 dollars based on U.S. Department of Commerce Implicit Price Deflator for Construction) (Eric S. Blake, 2007). In contrast, Hurricanes Harvey and Irma ravaged the states of Texas and Florida and the US territory of Puerto Rico in one year, 2017, and it has been estimated that the economic cost of these hurricanes will exceed \$150 billion (Horowitz, 2017). Is the occurrence of two or more costly hurricanes hitting the same US region within the same season a rare phenomenon? Or does it occur more often than not? To investigate, I will use the IBTrACKS (International Best Track Archive for Climate Stewardship) dataset to perform statistical analysis that will hopefully shed some light on these questions.

# Dataset

## Dataset description

The source dataset I will use comes from the IBTrACS (International Best Track Archive for Climate Stewardship) project: https://www.ncdc.noaa.gov/ibtracs/index.php. This project is endorsed by the WMO (World Meteorological Organization) as an "official archiving and distribution resource for tropical cyclone best track data". The IBTrACS project provides datasets that:

* Contains the most complete global set of historical tropical cyclones available
* Combines information from numerous tropical cyclone datasets
* Simplifies inter-agency comparisons by providing storm data from multiple sources in one place
* Provides data in popular formats to facilitate analysis
* Checks the quality of storm inventories, positions, pressures, and wind speeds, passing the information on to the user

I will be using the IBTrACS-WMO NetCDF file that contains all storms: https://www.ncdc.noaa.gov/ibtracs/index.php?name=wmo-data. Since NetCDF is a self-describing format, info about the variables contained in this dataset can be introspected. Additional info about the variables is located here: ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r10/wmo/netcdf/README.netcdf.

I will be performing some ETL (extraction, transformation and loading) tasks to prepare and filter (remove records with missing values) the source dataset to a tidy dataset which I will use for this analysis. The source dataset essentially aggregates every recorded tropical storm from different source agencies and provides time-series information of pertinent variables describing the storm as it progressed through its track.

The variables I will include from the source dataset are:

* **landfall** { True, False }
  ```
    short landfall(storm, time) ;
        landfall:long_name = "Minimum distance to land until next report (0=landfall)" ;
        landfall:units = "km" ;
        landfall:_FillValue = -999s ;
  ```
  * I will aggregate this variable into a single value of **True** or **False**. **True** signifies that the storm eventually made landfall at some point in the storm's track and **False** otherwise.
* **season** year based on season (INT)
  ```
    short season(storm) ;
          season:long_name = "Year based on season" ;
          season:Note = "Following WMO,\n",
                  "NH Seasons begin 1 January and \n",
                  "SH Seasons begin 1 July the prior year" ;
  ```
* **genesis_basin** { 0 = NA - North Atlantic, 1 = SA - South Atlantic, 2 = WP - West Pacific, 3 = EP - East Pacific, 4 = SP - South Pacific, 5 = NI - North Indian, 6 = SI - South Indian }
  ```
    byte genesis_basin(storm) ;
        genesis_basin:long_name = "Basin of genesis" ;
        genesis_basin:units = " " ;
        genesis_basin:key = "0 = NA - North Atlantic\n",
    "1 = SA - South Atlantic\n",
    "2 = WP - West Pacific\n",
    "3 = EP - East Pacific\n",
    "4 = SP - South Pacific\n",
    "5 = NI - North Indian\n",
    "6 = SI - South Indian\n",
    "7 = AS - Arabian Sea\n",
    "8 = BB - Bay of Bengal\n",
    "9 = EA - Eastern Australia\n",
    "10 = WA - Western Australia\n",
    "11 = CP - Central Pacific\n",
    "12 = CS - Carribbean Sea\n",
    "13 = GM - Gulf of Mexico\n",
    "14 = MM - Missing" ;
        genesis_basin:Note = "Based on where the storm began" ;
  ```
  * the additional variable info at ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r08/wmo/netcdf/README.netcdf states that only values 0-6 are used for this variable thus I will be discretizing the values for this variable
* **sub_basin** of first observation { 0 = NA - North Atlantic, 1 = SA - South Atlantic, 2 = WP - West Pacific, 3 = EP - East Pacific, 4 = SP - South Pacific, 5 = NI - North Indian, 6 = SI - South Indian, 7 = AS - Arabian Sea, 8 = BB - Bay of Bengal, 9 = EA - Eastern Australia, 10 = WA - Western Australia, 11 = CP - Central Pacific, 12 = CS - Carribbean Sea, 13 = GM - Gulf of Mexico, 14 = MM - Missing }
  ```
    byte sub_basin(storm, time) ;
        sub_basin:long_name = "Sub-Basin" ;
        sub_basin:units = " " ;
        sub_basin:key = "0 = NA - North Atlantic\n",
    "1 = SA - South Atlantic\n",
    "2 = WP - West Pacific\n",
    "3 = EP - East Pacific\n",
    "4 = SP - South Pacific\n",
    "5 = NI - North Indian\n",
    "6 = SI - South Indian\n",
    "7 = AS - Arabian Sea\n",
    "8 = BB - Bay of Bengal\n",
    "9 = EA - Eastern Australia\n",
    "10 = WA - Western Australia\n",
    "11 = CP - Central Pacific\n",
    "12 = CS - Carribbean Sea\n",
    "13 = GM - Gulf of Mexico\n",
    "14 = MM - Missing" ;
        sub_basin:Note = "Based on present location" ;
        sub_basin:_FillValue = '\201' ;
  ```
  * this variable will remain numeric since there are 14 values
* **time** of first observation (MJD value) (REAL)
  ```
    double time_wmo(storm, time) ;
        time_wmo:long_name = "Modified Julian Day" ;
        time_wmo:units = "days since 1858-11-17 00:00:00" ;
        time_wmo:_FillValue = 9.969209999999999e+36 ;
  ```
* **lon** (longitude) of first observation (REAL)
  ```
    short lon_wmo(storm, time) ;
        lon_wmo:long_name = "Storm center longitude" ;
        lon_wmo:units = "degrees_east" ;
        lon_wmo:scale_factor = 0.0099999998f ;
        lon_wmo:_FillValue = -32767s ;
  ```
* **lat** (latitude) of first observation (REAL)
  ```
    short lat_wmo(storm, time) ;
        lat_wmo:long_name = "Storm center latitude" ;
        lat_wmo:units = "degrees_north" ;
        lat_wmo:scale_factor = 0.0099999998f ;
        lat_wmo:_FillValue = -32767s ;
  ```
* **dist2land** (distance to land) of first observation (REAL)
  ```
    short dist2land(storm, time) ;
        dist2land:long_name = "Distance to land" ;
        dist2land:units = "km" ;
        dist2land:_FillValue = -999s ;
  ```
* **msw** (maximum sustained wind) of first observation (REAL)
  ```
    short pres_wmo(storm, time) ;
        pres_wmo:long_name = "Minimum Central Pressure (MCP)" ;
        pres_wmo:units = "mb" ;
        pres_wmo:scale_factor = 0.1f ;
        pres_wmo:_FillValue = -32767s ;
  ```
* **mcp** (minimum central pressure) of first observation (REAL)
  ```
    short wind_wmo(storm, time) ;
        wind_wmo:long_name = "Maximum Sustained Wind (MSW)" ;
        wind_wmo:units = "kt" ;
        wind_wmo:scale_factor = 0.1f ;
        wind_wmo:_FillValue = -32767s ;
  ```
* **nature** (storm nature) { 0 = TS - Tropical, 1 = SS - Subtropical, 2 = ET - Extratropical, 3 = DS - Disturbance, 4 = MX - Mix of conflicting reports, 5 = NR - Not Reported, 6 = MM - Missing, 7 = - Missing }
  ```
        nature_wmo:long_name = "Storm nature" ;
        nature_wmo:key = "0 = TS - Tropical\n",
    "1 = SS - Subtropical\n",
    "2 = ET - Extratropical\n",
    "3 = DS - Disturbance\n",
    "4 = MX - Mix of conflicting reports\n",
    "5 = NR - Not Reported\n",
    "6 = MM - Missing\n",
    "7 =  - Missing" ;
        nature_wmo:Note = "Based on classification from original sources" ;
        nature_wmo:_FillValue = '\201' ;
  ```
  * this variable will be discretized
* **track_type** { 0 = main - cylclogenesis to cyclolysis, 1 = merge - cyclogenesis to merger, 2 = split - split to cyclolysis, 3 = other - split to merger }
  ```
    byte track_type(storm) ;
        track_type:long_name = "Track type" ;
        track_type:key = "0 = main - cyclogenesis to cyclolysis\n",
    "1 = merge - cyclogenesis to merger\n",
    "2 = split - split to cyclolysis\n",
    "3 = other - split to merger" ;
  ```
  * this variable will be discretized


## Source dataset

In [55]:
import os, sys, re, json, time
from copy import deepcopy
from subprocess import check_output
import netCDF4 as NC
import numpy as np
import pandas as pd
from ipyleaflet import Map, GeoJSON
from astropy.time import Time
from IPython.display import display, HTML, Markdown


# get netcdf dataset
source_file = "Allstorms.ibtracs_wmo.v03r10.nc"

## Tidy dataset

In [ ]:
# define dict for discrete features
disc_map = {
    "basin": {
         0: "NA", # North Atlantic
         1: "SA", # South Atlantic
         2: "WP", # West Pacific
         3: "EP", # East Pacific
         4: "SP", # South Pacific
         5: "NI", # North Indian
         6: "SI", # South Indian
         7: "AS", # Arabian Sea
         8: "BB", # Bay of Bengal
         9: "EA", # Eastern Australia
        10: "WA", # Western Australia
        11: "CP", # Central Pacific
        12: "CS", # Carribbean Sea
        13: "GM", # Gulf of Mexico
        14: "MM", # Missing
    },
    "nature": {
        0: "TS",  # Tropical
        1: "SS",  # Subtropical
        2: "ET",  # Extratropical
        3: "DS",  # Disturbance
        4: "MX",  # Mix of conflicting reports
        5: "NR",  # Not Reported
        6: "MM",  # Missing
        7: "MM2", # Also Missing
    },
    "track_type": {
        0: "main",  # cyclogenesis to cyclolysis
        1: "merge", # cyclogenesis to merger
        2: "split", # split to cyclolysis
        3: "other", # split to merger
    },
    "month": {
        1: "Jan",
        2: "Feb",
        3: "Mar",
        4: "Apr",
        5: "May",
        6: "Jun",
        7: "Jul",
        8: "Aug",
        9: "Sep",
       10: "Oct",
       11: "Nov",
       12: "Dec",
    }
}


# conversion factors
kt_to_mph = 1.15078


def get_category(mph):
    """Return storm category from mph value of MSW."""
    if mph > 155.: return 5
    elif mph >= 131.: return 4
    elif mph >= 111.: return 3
    elif mph >= 96.: return 2
    elif mph >= 74.: return 1
    else: return 0
    
    
def get_tidy_dataset(source_ds_file, tidy_ds_file):
    """Return tidy dataset as a data frame. Save tidy dataset
    as an HDF5 file for re-use without having to tidy again."""
    
    # open source file
    ds = NC.Dataset(source_ds_file)
    
    # extract features from each hurricane and save into a list of dicts
    data = []
    landfall_count = 0
    
    # compile regular expression for matching unnamed storms
    unnamed_re = re.compile(r'(UNNAMED|NOT NAMED)')
    
    for i in range(ds.dimensions['storm'].size):
        
        # get number of observations
        obs = ds.variables['numObs'][i]
        if obs <= 2: continue # skip if there are 2 or less observations
        
        # get storm id (storm names can be re-used so we need to track them uniquely)
        id = np.array_str(NC.chartostring(ds.variables['storm_sn'][i,:]))[2:-1]
        
        # extract filterable features first
        name = np.array_str(NC.chartostring(ds.variables['name'][i,:]))[2:-1]
        season = ds.variables['season'][i]
        genesis_basin = ds.variables['genesis_basin'][i]
        
        # extract the landfall feature: if at any time in the storm's track
        # makes landfall, then the feature landfall == True; otherwise it
        # will be landfall == False
        landfall = (ds.variables['landfall'][i,:obs-1] == 0).any()
        
        # get max MSW (maximum sustained wind) for the storm and its category
        wind_wmo = ds.variables['wind_wmo'][i] * kt_to_mph # convert kt to mph
        max_msw_idx = wind_wmo.argmax()
        max_msw = wind_wmo[max_msw_idx]
        max_cat = get_category(max_msw)
        
        # get max MSW (maximum sustained wind) for the storm over land if reached landfall
        # and its max category; determine the summary_idx which will be used to summarize
        # the hurricane as a single data record
        masked_msw = np.ma.masked_where(ds.variables['landfall'][i] != 0.0, wind_wmo)
        masked_msw.set_fill_value(ds.variables['wind_wmo']._FillValue)
        max_land_msw_idx = masked_msw.argmax()
        if landfall:
            max_land_msw = wind_wmo[max_land_msw_idx]
            summary_idx = max_land_msw_idx
        else:
            max_land_msw = 0.0
            summary_idx = max_msw_idx
        max_land_cat = get_category(max_land_msw)
            
        # extract other filterable features
        sub_basin = ds.variables['sub_basin'][i,:]
        nature = ds.variables['nature_wmo'][i,:]
        
        # skip records that have missing values in features
        if genesis_basin == 14:
            continue
        # skipping this filter; this filters out the east pacific storms
        #if sub_basin[summary_idx] == 14:
        #    continue
        if nature[summary_idx] in (4, 5, 6, 7):
            continue
            
        # skip records with unnamed storms
        #if unnamed_re.search(name): continue
        
        # extract the rest of the features
        time_wmo = ds.variables['time_wmo'][i,:]
        time_iso = Time(time_wmo, format='mjd', scale='utc')
        
        # including the time feature as-is (absolute value) from the source doesn't make 
        # sense for prediction; a better feature to derive from the time feature is the 
        # month of year since this can give the algorithm insight into seasonal effects
        month = time_iso[summary_idx].datetime.month
        
        # extract lon and handle wrapping issue
        lon = ds.variables['lon_wmo'][i,:]
        lon_diff = lon[0] - lon[-1]
        if lon_diff > 180.:
            lon[np.where(lon > 0)] -= 360.
        elif lon_diff < 180.:
            lon[np.where(lon < 0)] += 360.
            
        # extract other features
        lat = ds.variables['lat_wmo'][i,:]
        dist2land = ds.variables['dist2land'][i,:]
        msw = ds.variables['wind_wmo'][i,:]
        mcp = ds.variables['pres_wmo'][i,:]
        tt = ds.variables['track_type'][i]
        
        # create GeoJSON of storm track
        ls = { 
            "type": "LineString",
            "coordinates": np.dstack((lon, lat))[0].tolist(),
        }
        
        # create feature for leaflet display;
        # stuff features into a message info for on_hover display
        msg = "{} {} {} {} {} {} {} {} {} {} {} {} {}".format(i, name, obs, genesis_basin, sub_basin[summary_idx],
                                                              time_iso[summary_idx].iso, lon[summary_idx], 
                                                              lat[summary_idx], dist2land[summary_idx],
                                                              msw[summary_idx], mcp[summary_idx], 
                                                              nature[summary_idx], tt, landfall)
        ls_feature = { 
            "type": "Feature",
            "properties": { "msg": msg },
            "geometry": ls,
        }
        
        # create data dict
        data.append({
            "id": id,
            "name": name,
            "genesis_basin": disc_map['basin'][genesis_basin],
            "sub_basin": disc_map['basin'][sub_basin[summary_idx]],
            "season": season,
            "time": time_wmo[summary_idx],
            "month": disc_map['month'][month],
            "lon": lon[summary_idx],
            "lat": lat[summary_idx],
            "dist2land": dist2land[summary_idx],
            "msw": msw[summary_idx],
            "mcp": mcp[summary_idx],
            "nature": disc_map['nature'][nature[summary_idx]],
            "track_type": disc_map['track_type'][tt],
            "landfall": landfall,
            "feature": json.dumps(ls_feature),
            "max_msw_mph": max_msw,
            "max_cat": max_cat,
            "max_land_msw_mph": max_land_msw,
            "max_land_cat": max_land_cat,
        })
        
        # tally landfall
        if landfall: landfall_count += 1
        
    # create data frame
    df = pd.DataFrame(data)
    
    # save data frame to HDF5
    df.to_hdf(tidy_ds_file, "tropicalstorms", format="table", complib="zlib", complevel=9)
    
    # return data frame
    return df

# tidy data file
tidy_ds_file = "Allstorms.ibtracs_wmo.v03r10-tidy.h5"

# get tidy dataframe
if os.path.exists(tidy_ds_file):
    df = pd.read_hdf(tidy_ds_file)
else:
    df = get_tidy_dataset(source_file, tidy_ds_file)

# print info
display(Markdown("### landfall distribution of tiday dataset"))
display(Markdown("* total storms: {}".format(len(data))))
display(Markdown("* total storms with variable landfall == True: {}".format(landfall_count)))
display(Markdown("* total storms with variable landfall == False: {}".format(len(data)-landfall_count)))